# Clip OSMFile using OSMOSIS

This method uses OSMOSIS to get an OSM extract from a big OSM File

### Parameters: ###

**INTEREST_AREA** - Extent of your interest area. Could be a path to a file (e.g. ESRI Shapefile or KML), a wkt string, or a list with wkt strings:

```Python
# If it is WKT
INTEREST_AREA = "POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))"

# OR if it is a List of WKT
# The list order should be: left; right; bottom; top
INTEREST_AREA = ["POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))", "POLYGON ((15 5, 20 20, 10 20, 5 10, 15 5))"]

# OR if it is a File
INTEREST_AREA = '/path/to/yourboundary.shp'
```

**OSMDATA** - Path to the output file that will have OSM Data available in the Interest Area;

**EPSG** - EPSG Code of the Spatial Reference System of INTEREST_AREA.

In [ ]:
from glass.gt.gop.osm.ovly import osm_extraction

INTEREST_AREA = '/mnt/d/gwork/prog_webgis/java_ex/dem_coimbra.tif'
OSM_FILE      = '/mnt/d/gwork/osm2lulc_imgcls/osmdata/portugal-latest.osm.pbf'
OSM_EXTRACT   = '/mnt/d/gwork/prog_webgis/java_ex/osm_coimbra.xml'

osm_extraction(INTEREST_AREA, OSM_FILE, OSM_EXTRACT)

# Convert OSM data into Feature Class #

In [ ]:
from glass.gt.toshp.osm import osm_to_featcls

osm_to_featcls(
    '/mnt/d/gwork/prog_webgis/java_ex/osm_coimbra.xml',
    '/mnt/d/gwork/prog_webgis/java_ex', useXmlName=True, outepsg=3763)

# OSM File to PSQL DB

In [ ]:
from glass.gql.to.osm import osm_to_psql

inOsm = '/mnt/d/gpto/osm2lulc_imgcls/osmdata/portugal-latest.osm.pbf'
out_db = 'osm_pt'

osm_to_psql(inOsm, out_db)

# OSM File to Relation Sctruture: #

In [ ]:
from glass.geo.gql.to.osm import osm_to_relationaldb

inOsm = '/home/osmtolulc/xpto/expvgi-inescc/api/osmapi/osmfiles/europe/portugal-latest.osm.pbf'

inSchema = {
    "TBL" : ['points', 'lines', 'multipolygons'],
    'FID' : 'ogc_fid',
    "COLS" : [
        'name', 'osm_id',
        #"ST_X(wkb_geometry) AS longitude",
        #"ST_Y(wkb_geometry) AS latitude",
        "wkb_geometry AS geom",
        "NULL AS featurecategoryid",
        "NULL AS flainarcategoryid",
        "NULL AS createdby",
        "NOW() AS createdon",
        "NULL AS updatedon",
        "NULL AS deletedon"
    ],
    "NOT_KEYS" : [
        'ogc_fid', 'osm_id', 'name', "wkb_geometry",
        'healthcare2', 'other_tags', 'osm_way_id',
        'ref', 'sidewalk', 'z_order', 'is_in', 'cuisine',
        'barrier', 'busway'
    ]
}

osmGeoTbl = {
    "points" : {"TBL" : 'osm_position', "FID" : 'positionid'},
    "multipolygons" : {"TBL" : "osm_polygons", "FID" : 'polygonid'},
    "lines" : {"TBL" : 'osm_lines', "FID" : 'lineid'}
}

osmCatTbl = {
    "TBL" : 'osmcategory', "FID" : "osmcategoryid",
    "KEY_COL" : "key", "VAL_COL" : "value",
    "COLS" : [
        "NULL AS createdby", "NOW() AS createdon",
        "NULL AS updatedon", "NULL AS deletedon"
    ]
}

osmRelTbl = {
    "points" : {"TBL" : "position_osmcat", "FID" : 'pososmcatid'},
    "multipolygons" : {"TBL" : "polygons_osmcat", "FID" : 'polygoncatid'},
    "lines" : {"TBL" : "lines_osmcat", "FID" : 'linecatid'},
}

outsql = '/home/osmtolulc/mrgis/flainar_osm_pt.sql'

osm_to_relationaldb(inOsm, inSchema, osmGeoTbl, osmCatTbl, osmRelTbl, outSQL=outsql)